# Packages

In [11]:
import requests
from scrapy import Selector
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests
import json
import pandas as pd

# 1.a Scraper composition du gouvernement

## Paramètres

In [8]:
URL_COMPOSITION_GOUVERNEMENT = "https://www.info.gouv.fr/composition-du-gouvernement"

In [9]:
html = requests.get(URL_COMPOSITION_GOUVERNEMENT).content

In [10]:
# Voir les 5000 premiers caractères du HTML
print(html.decode('utf-8')[:5000])

# Chercher si "h3" existe dans le HTML
print("h3" in str(html))
print("fr-card__title" in str(html))

<!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,"Helvetica Neue",Arial,"Noto Sans",sans-serif,"Apple Color Emoji","Segoe UI Emoji","Segoe UI Symbol","Noto Color Emoji"}body{display:flex;flex-direction:column;height:100vh;min-height:100vh}.main-content{margin:8rem auto;padding-left:1.5rem;max-width:60rem}@media (width <= 720px){.main-content{margin-top:4rem}}.h2{line-height:2.25rem;font-size:1.5rem;font-weight:500}@media (width <= 720px){.h2{line-height:1.5rem;font-size:1.25rem}}#challenge-error-text{background-image:url("");background-repeat:no-rep

--> "h3" ne s'affiche pas dans le HTML --> il faut exécuter le JS.

In [18]:
driver = webdriver.Firefox()
driver.get(URL_COMPOSITION_GOUVERNEMENT)

# Attendre le chargement
time.sleep(3)

html = driver.page_source

sel = Selector(text=html)

## Noms des ministres

In [ ]:
noms_ministres = sel.xpath('//h3[@class="fr-card__title"]//a[contains(@href, "personnalite")]/text()').extract()

# Nettoyage : suppression des espaces et des "\n"
noms_ministres = [nom.strip() for nom in noms_ministres]
noms_ministres

['Sébastien Lecornu',
 'Laurent Panifous',
 'Maud Bregeon',
 'Aurore Bergé',
 'Laurent Nunez',
 'Marie-Pierre Vedrenne',
 'Catherine Vautrin',
 'Alice Rufo',
 'Jean-Pierre Farandou',
 'Monique Barbut',
 'Catherine Chabaud',
 'Mathieu Lefèvre',
 'Gérald Darmanin',
 'Roland Lescure',
 'Sébastien Martin',
 'Anne Le Henanff',
 'Serge Papin',
 'Annie Genevard',
 'Édouard Geffray',
 'Jean-Noël Barrot',
 'Benjamin Haddad',
 'Nicolas Forissier',
 'Éléonore Caroit',
 'Rachida Dati',
 'Stéphanie Rist',
 'Charlotte Parmentier-Lecocq',
 'Naïma Moutchou',
 'Françoise Gatel',
 'Michel Fournier',
 'Amélie de Montchalin',
 'David Amiel',
 'Philippe Baptiste',
 'Marina Ferrari',
 'Philippe Tabarot',
 'Vincent Jeanbrun']

## Fonctions

In [ ]:
fonctions_ministres = sel.xpath('//p[@class="fr-card__desc"]/text()').extract()

# Nettoyage : suppression des espaces et des "\n"
fonctions_ministres = [fonction.strip() for fonction in fonctions_ministres]
fonctions_ministres

['Premier ministre',
 'Ministre délégué auprès du Premier ministre, chargé des Relations avec le Parlement',
 'Ministre déléguée auprès du Premier ministre, porte-parole du Gouvernement',
 "Ministre déléguée auprès du Premier ministre, chargée de l'Égalité entre les femmes et les hommes et de la Lutte contre les discriminations",
 'Ministre de l’Intérieur',
 'Ministre déléguée auprès du ministre de l’Intérieur',
 'Ministre des Armées et des Anciens combattants',
 'Ministre déléguée auprès de la ministre des Armées et des Anciens combattants',
 'Ministre du Travail et des Solidarités',
 'Ministre de la Transition écologique, de la Biodiversité et des Négociations internationales sur le climat et la nature',
 'Ministre déléguée auprès de la ministre de la Transition écologique, de la Biodiversité et des Négociations internationales sur le climat et la nature, chargée de la Mer et de la Pêche',
 'Ministre délégué auprès de la ministre de la Transition écologique, de la Biodiversité et des

# Scrapers API
Exploration de toutes les sources dispo par API sur data.gouv.fr :  
- https://www.data.gouv.fr/datasets/repertoire-national-des-elus-1/reuses_and_dataservices  
- Exemple liste députés avec liste déroulante pour choisir les instances en haut : https://explore.data.gouv.fr/fr/datasets/5c34c4d1634f4173183a64f1/#/resources/1ac42ff4-1336-44f8-a221-832039dbc142  

**A priori max page = 200**

## URLs des différentes données

In [ ]:
# Députées 1a
URL_DEPUTEES = f"https://tabular-api.data.gouv.fr/api/resources/1ac42ff4-1336-44f8-a221-832039dbc142/data/?page=2&page_size=10"

# Conseillères départementales 6a
URL_CONSEILLERES_DEPARTEMENTALES = f"https://tabular-api.data.gouv.fr/api/resources/601ef073-d986-4582-8e1a-ed14dc857fba/data/"

# Conseillères régionales 7a
URL_CONSEILLERES_REGIONALES = f"https://tabular-api.data.gouv.fr/api/resources/430e13f9-834b-4411-a1a8-da0b4b6e715c/data/"

# Sénatrices 3a
URL_SENATRICES = f"https://tabular-api.data.gouv.fr/api/resources/b78f8945-509f-4609-a4a7-3048b8370479/data/"

# Maires 5b
URL_MAIRES = f"https://tabular-api.data.gouv.fr/api/resources/2876a346-d50c-4911-934e-19ee07b0e503/data/"

In [88]:
# Choix de la donnée à récupérer à partir de l'URL
URL = URL_SENATRICES

## Requête API

In [ ]:
response = requests.get(
        url = URL,
        timeout=20
        )

if response.status_code == 200:
    try:
        resultat_api = json.loads(response.text)
    except json.JSONDecodeError as exc:
        print("Impossible de décoder le JSON :", exc)
        raise
    
data = resultat_api['data']
data = pd.DataFrame(data)
data

,__id,Code du département,Libellé du département,Code de la collectivité à statut particulier,Libellé de la collectivité à statut particulier,Nom de l'élu,Prénom de l'élu,Code sexe,Date de naissance,Code de la catégorie socio-professionnelle,Libellé de la catégorie socio-professionnelle,Date de début du mandat
0,1,01,Ain,None,None,BLATRIX CONTAT,Florence,F,1966-03-30,34,"Professeur, profession scientifique",2020-01-10
1,2,01,Ain,None,None,CHAIZE,Patrick,M,1963-03-22,85,Personne diverse sans activité professionnelle...,2020-01-10
2,3,01,Ain,None,None,GOY-CHAVENT,Sylvie,F,1963-05-23,34,"Professeur, profession scientifique",2020-01-10
3,4,02,Aisne,None,None,GRUNY,Pascale,F,1960-02-18,37,Cadre administratif et commercial d'entreprise,2020-01-10
4,5,02,Aisne,None,None,LEFEVRE,Antoine,M,1966-02-18,74,Ancien cadre,2020-01-10
5,6,02,Aisne,None,None,VERZELEN,Pierre-Jean,M,1983-08-29,85,Personne diverse sans activité professionnelle...,2020-01-10
6,7,03,Allier,None,None,MALHURET,Claude,M,1950-08-03,74,Ancien cadre,2020-01-10
7,8,03,Allier,None,None,ROJOUAN,Bruno,M,1957-06-03,74,Ancien cadre,2020-01-10
8,9,04,Alpes-De-Haute-Provence,None,None,ROUX,Jean-Yves,M,1970-01-18,77,Ancien employé,2020-01-10
9,10,05,Hautes-Alpes,None,None,ARNAUD,Jean-Michel,M,1966-04-28,11,Agriculteur sur petite exploitation,2020-01-10


## Conservation des données qui nous intéressent
Nom, prénom, sexe, date début mandat, localisation, parti politique

In [94]:
df_traite = data[["Code du département", "Libellé du département", "Nom de l'élu", "Prénom de l'élu", "Code sexe", "Date de début du mandat"]]
df_traite

,Code du département,Libellé du département,Nom de l'élu,Prénom de l'élu,Code sexe,Date de début du mandat
0,01,Ain,BLATRIX CONTAT,Florence,F,2020-01-10
1,01,Ain,CHAIZE,Patrick,M,2020-01-10
2,01,Ain,GOY-CHAVENT,Sylvie,F,2020-01-10
3,02,Aisne,GRUNY,Pascale,F,2020-01-10
4,02,Aisne,LEFEVRE,Antoine,M,2020-01-10
5,02,Aisne,VERZELEN,Pierre-Jean,M,2020-01-10
6,03,Allier,MALHURET,Claude,M,2020-01-10
7,03,Allier,ROJOUAN,Bruno,M,2020-01-10
8,04,Alpes-De-Haute-Provence,ROUX,Jean-Yves,M,2020-01-10
9,05,Hautes-Alpes,ARNAUD,Jean-Michel,M,2020-01-10
